In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
from mpl_toolkits.axes_grid1 import make_axes_locatable
import progressbar

plt.ioff()

# parametres
mx = 2001      # maillage x
my = 201       # maillage y
Db = 0.004     # diametre bulle
pas_t = 0.005  # pas de temps (e-6 s)
nmax = 47000   # nombre d'iteration
kres = 100     # frequence de sortie des residus
M = 1.24       # Mach

Uflow = {'1.18':93.6482,
         '1.24':125.1538,
         '1.73':333.5403,
         '2.00':433.8496,
         '3.00':771.1353,
         '4.00':1084.2369}

U = Uflow[f'{M:.2f}']        # m/s

n = int(nmax / kres)  # number of files

mx = mx - 4  # mx - 2 results in x and we'll exclude the two last ones (outliers)
my = my - 2  # my - 2 results in y
t = kres * pas_t  # temps entre images
t0 = - 0.001/(346.956*M) * 1e6    # initial time at the interaction choc/bubble
T = Db / U * 1e6    # constant of time
print(t0, t, t/T, 346.956*M)

-2.324362780592426 0.5 0.015644225 430.22544


M = 1.18:
    nmax = 77000
    kres = 90
    t/T = 0.010535422499999999
M = 1.73:
    nmax = 24000
    kres = 25
    t/T = 0.010423134374999999
M = 2.00:
    nmax = 17252
    kres = 19
    t/T = 0.010303928
M = 3.00:
    nmax = 9790
    kres = 11
    t/T = 0.010603110375
M = 4.00:
    nmax = 69966
    kres = 78
    t/T = 0.010571309775000001

In [9]:
layes = np.array([-45,25,95,165,235,305,375,445,585,655,795,1005,1145,1285,1425,1565,1705,1845,1985,2125])
Dlayes = 0.038
layes = layes / (Dlayes / 125.1538 * 1e6)
print(layes)

[-0.14820845  0.08233803  0.3128845   0.54343097  0.77397745  1.00452392
  1.23507039  1.46561687  1.92670982  2.15725629  2.61834924  3.30998866
  3.77108161  4.23217455  4.6932675   5.15436045  5.61545339  6.07654634
  6.53763929  6.99873224]


### Visualisation 2D

In [10]:
parameters = ["alpha", "Pstat", "rho", "gradrho", "u", "v"]

vmin = {}
vmax = {}

for parameter in parameters:
    vmin[parameter] = np.inf
    vmax[parameter] = -np.inf
    
units = {'alpha':'',
         'Pstat':'bar',
         'rho':'kg/m³',
         'gradrho':'kg/m³',
         'u':'m/s',
         'v':'m/s'}

#### Finding the global min and max for each parameter to scale the graphs

In [11]:
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    for parameter in parameters:
        local_min = data[f'{parameter}'].min()
        local_max = data[f'{parameter}'].max()
        if local_min < vmin[f'{parameter}']:
            vmin[f'{parameter}'] = local_min
        if local_max > vmax[f'{parameter}']:
            vmax[f'{parameter}'] = local_max
    
    bar.update(i)

100% (440 of 440) |######################| Elapsed Time: 0:04:58 ETA:  00:00:00

#### $\rho$ min and max among all Machs for comparison

In [7]:
vmin = {'rho':0.160}
vmax = {'rho':6.000}  # 1.967, 4.100, 5.000, 6.688, 8.215

#### Generate images

Report (adequate font sizes)

In [ ]:
parameters = ["alpha", "Pstat", "rho"]

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Individual images for each parameter
    
    for parameter in parameters:
        variable = data[f'{parameter}'].to_numpy(copy=True)
        variable = variable.reshape((my,mx))
        
        plt.figure(figsize=(16,4))
        plt.rcParams.update({'font.size': 16})
        ax = plt.subplot()
        plt.title(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
        im = ax.imshow(variable, origin='lower', cmap='plasma',
                       extent=[data['x'].min(),
                               data['x'].max(),
                               data['y'].min(),
                               data['y'].max()],
                       vmin=vmin[f'{parameter}'],
                       vmax=vmax[f'{parameter}'])

        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)

        ticks=[(5-i)/5*vmin[f'{parameter}']+i/5*vmax[f'{parameter}'] for i in range(6)]
        plt.colorbar(im, cax=cax, ticks=ticks).set_label(units[f'{parameter}'])

        plt.savefig(f'Images/Report/vis2D_{parameter}_t{i*t:05.1f}.png', dpi=180)
        plt.close()

Normal

In [ ]:
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Individual images for each parameter
    
    for parameter in parameters:
        variable = data[f'{parameter}'].to_numpy(copy=True)
        variable = variable.reshape((my,mx))
        
        plt.figure(figsize=(16,5))  # (16,4) para tamanho normal
        ax = plt.subplot()
        plt.title(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
        im = ax.imshow(variable, origin='lower', cmap='plasma',
                       extent=[data['x'].min(),
                               data['x'].max(),
                               data['y'].min(),
                               data['y'].max()],
                       vmin=vmin[f'{parameter}'],
                       vmax=vmax[f'{parameter}'])

        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)

        ticks=[(7-i)/7*vmin[f'{parameter}']+i/7*vmax[f'{parameter}'] for i in range(8)]
        plt.colorbar(im, cax=cax, ticks=ticks).set_label(units[f'{parameter}'])

        plt.savefig(f'Images/vis2D_{parameter}_t{i*t:05.1f}.png', dpi=180)
        plt.close()
        
    bar.update(i)

***
#### Two parameters together

#### $\alpha$ and $P_{stat}$

In [ ]:
parameter1 = 'alpha'
parameter2 = 'Pstat'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}_maillage_1001x101/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5)) #(16,5)
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)  # -0.0 bigger domaine
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'Images/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

#### $\rho$ and $\nabla \rho$

In [ ]:
parameter1 = 'rho'
parameter2 = 'gradrho'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}_maillage_1001x101/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5))
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'Images/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

#### $u$ and $v$

In [ ]:
parameter1 = 'u'
parameter2 = 'v'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}_maillage_1001x101/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5))
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'Images/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

#### $\rho$ and $P_{stat}$

In [ ]:
parameter1 = 'rho'
parameter2 = 'Pstat'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5))
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'Images/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

### Complete bubble

In [14]:
parameter = 'rho'
bar = progressbar.ProgressBar(max_value=n)

for i in range(440,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = np.flip(par1, axis=0)
    
    variable = np.append(par2, par1, axis=0)
    
    # Plot

    plt.figure(figsize=(16,5))
    plt.rcParams.update({'font.size': 16})
    ax = plt.subplot()
    ax.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    plt.title(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    im = ax.imshow(variable, origin='lower', cmap='plasma',
                   extent=[data['x'].min(),
                           data['x'].max(),
                           -data['y'].max(),
                           data['y'].max()],
                   vmin=vmin[f'{parameter}'],
                   vmax=vmax[f'{parameter}'])
    
    ## Colorbar
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter}']+i/7*vmax[f'{parameter}'] for i in range(8)]
    plt.colorbar(im, cax=cax, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter}'])

    ## Saving
    
    plt.savefig(f'Images/vis2D_complet_{parameter}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

100% (470 of 470) |######################| Elapsed Time: 0:00:28 ETA:  00:00:00

### Generate gifs

In [ ]:
parameters = ["complet_rho"] #"alpha", "Pstat", "rho", "alpha&Pstat", "rho&Pstat"]

filenames = {}

for parameter in parameters:
    filenames[parameter] = []

for i in range(1,n+1):
    for parameter in parameters:
        filenames[f'{parameter}'].append(f'vis2D_domain_comparison_t{i*t:05.1f}.png')
        
for parameter in parameters:
    bar = progressbar.ProgressBar(max_value=len(filenames[f'{parameter}']))
    with imageio.get_writer(f'Images/M{M:.2f}_domain_comparison.gif', mode='I', duration = 1/16) as writer:  # 1/24*U/Uflow['1.18']
        for filename in filenames[f'{parameter}']:
            image = imageio.imread(f'Images/{filename}')
            writer.append_data(image)
            bar.update(filenames[f'{parameter}'].index(filename)+1)

***
#### Test section

In [ ]:
    filename = 'SCB_output00031'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}_domaine_0.04x0.008/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    # alpha and rho
    
    alpha = data['alpha'].to_numpy(copy=True)
    alpha = alpha.reshape((my,mx))
    
    rho = data['rho'].to_numpy(copy=True)
    rho = rho.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,8))
    fig.suptitle(f'M = {M:.2f}, t = {i*t:.1f} $\mu$s')
    plt.subplots_adjust(hspace=0.0)
    
    # Subplot 1 - alpha
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(r'$\alpha$     ', rotation='horizontal', fontsize=12)
    im1 = ax1.imshow(alpha, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin['alpha'],
                     vmax=vmax['alpha'])
    
    ## Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin["alpha"]+i/7*vmax["alpha"] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks)
    cax1.xaxis.set_ticks_position("top")
    
    # Subplot 2 - rho
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(r'$\rho$     ', rotation='horizontal', fontsize=12)
    im2 = ax2.imshow(rho, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin['rho'],
                     vmax=vmax['rho'])
    ax2.invert_yaxis()
    
    ## Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin['rho']+i/7*vmax['rho'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label('kg/m3')

    
    plt.show()

In [ ]:
    filename = 'SCB_output00002'

    data = pd.read_csv(f'Output_SCB_M{M:.2f}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    # alpha and rho
    
    alpha = data['alpha'].to_numpy(copy=True)
    alpha = alpha.reshape((my,mx))
    
    rho = data['rho'].to_numpy(copy=True)
    rho = rho.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5))
    fig.suptitle(f'M = {M:.2f}, t = {i*t:.1f} $\mu$s')
    plt.subplots_adjust(hspace=0)
    
    # Subplot 1 - alpha
    im1 = ax1.imshow(alpha, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin['alpha'],
                     vmax=vmax['alpha'])
    
    ## Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin["alpha"]+i/7*vmax["alpha"] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks)
    cax1.xaxis.set_ticks_position("top")
    
    # Subplot 2 - rho
    ax2.tick_params(bottom=False, labelbottom='off', top=True, labeltop='on')
    im2 = ax2.imshow(rho, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin['rho'],
                     vmax=vmax['rho'])
    ax2.invert_yaxis()
    
    ## Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin['rho']+i/7*vmax['rho'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks)
    
    plt.show()

In [ ]:
data = pd.read_csv(f'Output_SCB_M{M:.2f}_domaine_0.04x0.008/SCB_output00030.data',
                   sep="\s+", header=None,
                   names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                   low_memory=False)

for column in data.columns:
    if data[column].dtypes != 'float64':
        data[column] = pd.to_numeric(data[column], errors='coerce')

data = data.replace(np.nan, 0)

data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)

data['x'] = data['x'] / Db
data['y'] = data['y'] / Db

data.head()

In [ ]:
Pstat = data['Pstat'].to_numpy(copy=True)
Pstat = Pstat.reshape((my,mx))
vmin=0.5#data['Pstat'].min()
vmax=1.6#data['Pstat'].max()

plt.figure(figsize=(16,5))
plt.rcParams.update({'font.size': 16})
ax = plt.subplot()
plt.title(f'test T = {T:.2f}')
im = ax.imshow(Pstat, origin='lower', cmap='plasma',
               extent=[data['x'].min(),
                       data['x'].max(),
                       data['y'].min(),
                       data['y'].max()],
               vmin=vmin, vmax=vmax)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

ticks=[(5-i)/5*vmin+i/5*vmax for i in range(6)]
plt.colorbar(im, cax=cax, ticks=ticks)

#plt.savefig('test.png', dpi=180)
plt.show()
plt.close()

In [ ]:
# Different way of plotting
ax = plt.subplot()
im = ax.scatter(data['x'], data['y'], c=data['Pstat'],
                marker='s', s=1, cmap='plasma')
plt.axis('equal')
plt.title(f't = {t} $\mu$s')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

plt.colorbar(im, cax=cax)
plt.show()

___
### Graphics

#### Tous les parametres

In [ ]:
parameters = ['alpha','bubble_len','cson','density','fractiony',
              'pmax','pressure','pwall_max','residu_final','temperature',
              'umax','velocity','vol_vap','wall_pressure']

for parameter in parameters:
    if parameter in ['bubble_len','pmax','pwall_max','residu_final','umax','vol_vap']:
        #filename = 'SCB_' + parameter

        #data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
        #data.drop([1999,2000], inplace=True)
        #data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
        #plt.savefig(f'{filename}.png')
        #plt.close()
        pass
    
    else:
        n = 51  # number of files
        filenames = []

        for i in range(1,n+1):
            filename = 'SCB_' + parameter
            if i != n:
                filename += f'{i:05}'
            else:
                filename += '_final'
            filenames.append(f'{filename}.png')

            data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
            data.drop(data.tail(2).index, inplace=True)
            data.plot(x='x', y=parameter, ylabel=parameter, legend=False)
            plt.savefig(f'Images/{filename}.png')
            plt.close()

        with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
            for filename in filenames:
                image = imageio.imread(f'Images/{filename}')
                writer.append_data(image)

#### alpha

In [ ]:
n = 51  # number of files
parameter = 'alpha'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### cson

In [ ]:
n = 51  # number of files
parameter = 'cson'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### density

In [ ]:
n = 51  # number of files
parameter = 'density'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### fractiony

In [ ]:
n = 51  # number of files
parameter = 'fractiony'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### pressure

In [ ]:
n = 51  # number of files
parameter = 'pressure'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### temperature

In [ ]:
n = 51  # number of files
parameter = 'temperature'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### velocity

In [ ]:
n = 51  # number of files
parameter = 'velocity'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)

#### wall_pressure

In [ ]:
n = 51  # number of files
parameter = 'wall_pressure'
filenames = []

for i in range(1,n+1):
    filename = 'SCB_' + parameter
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'
    filenames.append(f'{filename}.png')
    
    data = pd.read_csv(f'Output_SCB/{filename}.data', sep="\s+", header=None, names=['x',parameter])
    data.drop([1999,2000], inplace=True)
    data.plot(x='x',y=parameter,ylabel=parameter,legend=False)
    plt.savefig(f'Images/{filename}.png')
    plt.close()

with imageio.get_writer(f'{parameter}.gif', mode='I', duration = 0.1) as writer:
    for filename in filenames:
        image = imageio.imread(f'Images/{filename}')
        writer.append_data(image)